In [1]:
import os
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import nltk
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import pickle

In [2]:
table=pd.read_csv(os.path.dirname(os.getcwd())+'\\OUTPUTS\\INTERMEDIATE\\state_temp.csv', low_memory=False)

In [3]:
table['Own_Type'] =np.nan

In [4]:
null = table.loc[pd.isnull(table['Simple_Owners']) == True]
table = table.loc[~pd.isnull(table['Simple_Owners']) == True]

In [5]:
null['Own_Type'] = 45

In [6]:
table['Simple_Owners'] = table['Simple_Owners'].fillna('NoData')

In [7]:
#initial family 

In [8]:
family = table.loc[(table['initial_class'] == 1) | (table['initial_class'] == 3)]
other = table.loc[(table['initial_class'] != 1) & (table['initial_class'] != 3)]

In [9]:
trust_kw = [' trust ', ' rev tr of ']

In [10]:
trusts = other[other['Simple_Owners'].str.contains('|'.join(trust_kw))]
other = other[~other['Simple_Owners'].str.contains('|'.join(trust_kw))]

In [11]:
searchfor=['revocable', 'living', 'family', ' rev tr of', ' revoc trust', ' revoc ']
family_trusts= trusts[trusts['Simple_Owners'].str.contains('|'.join(searchfor))]
unknown_trusts= trusts[~trusts['Simple_Owners'].str.contains('|'.join(searchfor))]

In [12]:
#hard coding unk trusts
#other=pd.concat([other, unknown_trusts])
unknown_trusts['Own_Type'] = 41

<ipython-input-12-8392cf983357>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unknown_trusts['Own_Type'] = 41


In [13]:
family=pd.concat([family, family_trusts])

In [14]:
#farms

In [15]:
farms= other[other['Simple_Owners'].str.contains(" farms ")]
other= other[~other['Simple_Owners'].str.contains(" farms ")]

In [16]:
searchfor=[' family ', ' brother ', ' son ', ' daughter ']
family_farms= farms[farms['Simple_Owners'].str.contains('|'.join(searchfor))]
unknown_farms= farms[~farms['Simple_Owners'].str.contains('|'.join(searchfor))]

In [17]:
family=pd.concat([family, family_farms])

In [18]:
other=pd.concat([other, unknown_farms])

In [19]:
searchfor=[' family ', ' brother ', ' son ', ' daughter ']

other_family_language = other[other['Simple_Owners'].str.contains('|'.join([x.lower() for x in searchfor]))]
other= other[~other['Simple_Owners'].str.contains('|'.join([x.lower() for x in searchfor]))]

In [20]:
family=pd.concat([family, other_family_language])

In [21]:
#corprate

In [22]:
corp_keywords=[" BANK ", "CORPORATION", "PARTICIPATION", "PROPERTIES",
          "ASSOCIATIONS", "ASSOCIATES", " ASSOCI", "MAINTENANCE", 
               "MAINTENANC", "TELEPHONE","ELECTRIC", "ENTERPRISES", "ENTERPRISE", 
               "AUTHORITY", "HOMEOWNERS", "INTERNATIONAL", " OFFICE", "INVESTMENT", 
               " MGMT"," INN", "ASSOCIATION", "ACCOUNTING", "MAINTEN", "PRODUCT", "MUTUAL", 
         " ESTATES", " & SONS", "FINANCE", " TITLE", "WIRELESS", "COMMUNICATION", 
               " SERVICE", "BAR & GRILL", " DEPT", " CTR", " LOAN", "SPECIALTIES", "BRANDS", " UNITED"
         "CREDIT", " UNION", "CORPORATE", "TREASURER", " ADMIN", "UTILITIES", 
         "COMMERCIAL", " STORAGE", "REAL ESTATE", "CREDIT UNION"]  

biz_word_drop=[" CORP", " LLC", " INC", " LTD", " HRS", "MGT", " OF ", " CO ", " LP", " MGMT"
               " STE ", " L L C", " L P"]

In [23]:
#for component in map(lambda x:x.lower(),biz_word_drop):
      #  other_family_language['Simple_owners']=other_family_language['Simple_owners'].str.replace(component ,'')
#other_family_language_corp = other_family_language[other_family_language['Simple_owners'].str.contains('|'.join(map(lambda x:x.lower(),corp_keywords)))]
#other_family_language_family = other_family_language[~other_family_language['Simple_owners'].str.contains('|'.join(map(lambda x:x.lower(),corp_keywords)))]

In [24]:
corp = other[other['Simple_Owners'].str.contains('|'.join([x.lower() for x in corp_keywords]))]
other = other[~other['Simple_Owners'].str.contains('|'.join([x.lower() for x in corp_keywords]))]

In [25]:
#government

In [26]:
keywords=[" BANK ", " CORP", "PARTICIPATION", " TRUST", " TRUS", " CO ", " LP"
          "UNIVERSITY", "COLLEGE", " CHURCH"," CLUB",
          "BAPTISI", "EVANGELICAL", "METHODIST", "CATHOLIC", "PROPERTIES",
          "ASSOCIATIONS", "ASSOCIATES"," ASSOCI","SOCIETY", "MAINTENANCE", 
          "MAINTENANC", "ENTERPRISES", "ENTERPRISE", "AUTHORITY", "HOMEOWNERS",
         "INTERNATIONAL", "INVESTMENT", " HOME", " MGMT", " PRESBYTERIAN", " INN", "ASSOCIATION",
          " STE ", "ACCOUNTING", "MAINTEN", "PRODUCT", "MUTUAL", 
         "ESTATES", "PARTNER", " & SONS", "FINANCE", " TITLE", " L P ", " FARM",
         "WIRELESS", "COMMUNICATION", " SERVICE", "BAR & GRILL", " CTR ", " LOAN", "SPECIALTIES", "BRANDS",
         "CREDIT", "CORPORATE", "TREASURER", " ADMIN", "UTILITIES", "COMMERCIAL", " STORAGE", "FAMILY"]    

government_keywords=[' state ', ' county ', ' federal ', ' government ', ' city of ', ' town of ',
                     ' township ', ' department ', ' bureau ']

government = other[other['Simple_Owners'].str.contains('|'.join([x.lower() for x in government_keywords]))]
other = other[~other['Simple_Owners'].str.contains('|'.join([x.lower() for x in government_keywords]))]

In [27]:
gov=government[~government['Simple_Owners'].str.contains('|'.join([x.lower() for x in keywords]))]
unk_gov = government[government['Simple_Owners'].str.contains('|'.join([x.lower() for x in keywords]))]
other = pd.concat([other, unk_gov])

In [28]:
gov['Own_Type'] = 0

<ipython-input-28-5a4aad64b27b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gov['Own_Type'] = 0


In [29]:
#searchfor=[' family ', ' brother ', ' son ', ' daughter ']
#family_government= government[government['Simple_owners'].str.contains('|'.join(searchfor))]
#unknown_government= government[~government['Simple_owners'].str.contains('|'.join(searchfor))]

In [30]:
#religious

In [31]:
rel_key_words= [' Adventist ', ' Baptist ', ' Brethren ', ' Catholic ',
                   'Christian Church', ' Church of ', ' Episcopal ', ' Anglican ',
                   ' Lutheran ', ' Mennonite ', ' Methodist ', ' Moravian ', 'Mormon ', 
                  ' Nazarene ', ' Orthodox ', 'Pentecostal ', ' Presbyterian ', 
                  ' Reformed Church ', 'Spiritualist ', ' Mosques ', ' Islamic Center ',
                  ' Mosque ', ' Masjid ', ' Synagogues ', ' Temple ', ' Congregation '
               , ' of bethlehem ', 'kingdom of god', 'christian*church']


In [32]:
religious_groups = other[other['Simple_Owners'].str.contains('|'.join([x.lower() for x in rel_key_words]))]
other = other[~other['Simple_Owners'].str.contains('|'.join([x.lower() for x in rel_key_words]))]

In [33]:
religious_groups['Own_Type'] = 43

In [34]:
family['Own_Type'] = 45

In [35]:
ckw = ['Rayonier', 'Weyerhaeuser', 'Plum Creek'] 
corporate = other[other['Simple_Owners'].str.contains('|'.join([x.lower() for x in ckw]))]
other = other[~other['Simple_Owners'].str.contains('|'.join([x.lower() for x in ckw]))]

In [36]:
corporate['Own_Type'] = 41

In [37]:
kw42 = ['Nature Conservancy']
c42 = other[other['Simple_Owners'].str.contains('|'.join([x.lower() for x in kw42]))]
other = other[~other['Simple_Owners'].str.contains('|'.join([x.lower() for x in kw42]))]

In [38]:
c42['Own_Type'] = 42

In [39]:
kw43 = ['YMCA', 'YWCA', 'Boys Scouts', 'Girls Scouts']
c43 = other[other['Simple_Owners'].str.contains('|'.join([x.lower() for x in kw43]))]
other = other[~other['Simple_Owners'].str.contains('|'.join([x.lower() for x in kw43]))]

In [40]:
c43['Own_Type'] = 43

In [41]:
other['Simple_Owners'] = other.Simple_Owners.map(lambda x: str(x).lower())
other['Simple_Owners']= other.Simple_Owners.str.replace('[^\w\s]', '')
other['Simple_Owners'] = other['Simple_Owners'].apply(nltk.word_tokenize)

<ipython-input-41-3c01850ad120>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  other['Simple_Owners']= other.Simple_Owners.str.replace('[^\w\s]', '')


In [42]:
stemmer =PorterStemmer()
other['Simple_Owners']=other['Simple_Owners'].apply(lambda x: [stemmer.stem(y) for y in x])

In [43]:
other['Simple_Owners']=other['Simple_Owners'].apply(lambda x: ' '.join(x))

In [44]:
count_vect = CountVectorizer()
counts = count_vect.fit_transform(other['Simple_Owners'])

In [45]:
transformer = TfidfTransformer().fit(counts)

counts=transformer.transform(counts)

In [46]:
count_dict = {k: v for k, v in sorted(count_vect.vocabulary_.items(), key=lambda item: item[1])}

In [47]:
x=pd.DataFrame(counts.todense())

In [48]:
with open(os.path.dirname(os.getcwd())+'\\OUTPUTS\\model_dict.pkl', 'rb') as handle:
    model_dict = pickle.load(handle)

In [49]:
model_dict = {k: v for k, v in sorted(model_dict.items(), key=lambda item: item[1])}

In [50]:
model_df = pd.DataFrame(model_dict, index = [0])

In [51]:
model_df = model_df.drop(0)

In [52]:
from tqdm import tqdm
for i in tqdm(list(model_dict.keys())):
    if i in list(count_dict.keys()):
        model_df[i] = x.iloc[:,count_dict[i]]
    else:
        model_df[i] = pd.Series(np.zeros((len(x))))

100%|█████████████████████████████████████████████████████████████████████████████| 4810/4810 [00:08<00:00, 596.54it/s]


In [53]:
loaded_model=pickle.load(open(os.path.dirname(os.getcwd())+'\\OUTPUTS\\classify_unknown_ownership_model.pkl', 'rb'))

In [54]:
other['Own_Type']=loaded_model.predict(model_df)

In [55]:
total=pd.concat([other, unknown_trusts, family, gov, religious_groups, corporate, c42, c43, null, corp])

In [56]:
gov = total.loc[total['Own_Type'] == 0]
not_gov = total.loc[total['Own_Type'] != 0]

In [57]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

In [58]:
searchfor=['STATE ', gov['State'].unique()[0], {y:x for x,y in us_state_to_abbrev.items()}[gov['State'].unique()[0]]]
state_gov = gov[(gov['OWN1'].str.contains('|'.join(searchfor))) | (gov['OWN2'].str.contains('|'.join(searchfor)))]
other_gov = gov[~(gov['OWN1'].str.contains('|'.join(searchfor))) | (gov['OWN2'].str.contains('|'.join(searchfor)))]

In [59]:
searchfor = ['FEDERAL ', 'BUREAU', 'UNITED STATES ']
fed_gov = other_gov[(other_gov['OWN1'].str.contains('|'.join(searchfor))) | (other_gov['OWN2'].str.contains('|'.join(searchfor)))]
local_gov = other_gov[~(other_gov['OWN1'].str.contains('|'.join(searchfor))) | (other_gov['OWN2'].str.contains('|'.join(searchfor)))]

In [60]:
fed_gov['Own_Type'] = 25
state_gov['Own_Type'] = 31
local_gov['Own_Type'] = 32

<ipython-input-60-a8677e3caf35>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fed_gov['Own_Type'] = 25
<ipython-input-60-a8677e3caf35>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_gov['Own_Type'] = 31
<ipython-input-60-a8677e3caf35>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

In [61]:
total=pd.concat([fed_gov, state_gov, local_gov, not_gov])

In [67]:
total = total.drop_duplicates(['PRCLDMPID'], keep='first')

In [68]:
total.to_csv(os.path.dirname(os.getcwd())+'\\OUTPUTS\\INTERMEDIATE\\classified_state_temp.csv')

In [ ]:
print('Ownerships Classified')